# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [84]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [86]:
# 1) Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Município'
r = requests.get(url)
r.status_code

200

In [87]:
data_json = r.json()

In [88]:
df_geral = pd.DataFrame(data_json['items'])

In [89]:
df_pvl_rj = df_geral[df_geral["uf"] == 'RJ']

In [90]:
# 2) Quais são os três status das solicitações mais frequentes na base? Quais são suas frequências?
df_pvl_rj['status'].value_counts()[:3]

status
Deferido        106
Arquivado        53
Regularizado     26
Name: count, dtype: int64

In [91]:
# 3) Construa uma nova variável que contenha o ano do status. Observe que data_status vem como tipo object no DataFrame. 
#Dica: você pode usar o método .str para transformar o tipo da variável em string, em seguida um método como slice() ou split().

selecao = df_pvl_rj.data_status
ano_status = selecao.str.split('/' ,expand=True)
ano_status = ano_status.loc[:, 2]
ano_status

0      2013
1      2013
2      2012
3      2009
4      2015
       ... 
263    2023
264    2019
265    2012
266    2014
267    2013
Name: 2, Length: 268, dtype: object

In [92]:
#4) Indique a frequência de cada ano do campo construído no item (3).
ano = pd.Series(ano_status)
ano.value_counts()

2
2008    37
2007    29
2023    28
2014    20
2010    19
2019    16
2013    15
2020    15
2012    13
2022    11
2024    10
2011    10
2009     9
2015     8
2017     5
2021     5
2016     4
2018     4
2002     4
2006     4
2003     1
2004     1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

<span style="color: purple;"> Comparando o URL da aula ('http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=SP&tipo_interessado=Estado') com o URL da atividade ('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Município'), podemos notar que é o final que diferencia o resultado. ```uf``` e ```tipo_interessdo```. </span>

In [156]:
#1) Uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado ('Estado'ou Município), 
#e que devolve os dados da consulta no formato DataFrame.

def consulta_pvl(uf, tipo_interessado):
 # Montando a URL da consulta com base nos parâmetros
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}'

    # Fazendo a requisição
    response = requests.get(url)
    data = response.json()

    # Convertendo os dados para DataFrame
    df = pd.DataFrame(data['items'])

    return df

In [158]:
# 2) Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com status em 'Arquivado por decurso de prazo' estão registradas?
#Acessando os dados sobre MG
df_mg = consulta_pvl('MG', 'Estado')

# Filtrar os dados com status 'Arquivado por decurso de prazo'
arquivados_mg = df_mg[df_mg['status'] == 'Arquivado por decurso de prazo']

# Contar quantas solicitações estão com esse status
quantidade_arquivados = len(arquivados_mg)
print(quantidade_arquivados)


1


In [208]:
# 3) Qual é o município da Bahia com mais solicitações deferidas?

df_ba_mun = consulta_pvl('BA', 'Município')
deferidos_ba = df_ba[df_ba_mun['status'] == 'Deferido']
print(f'O município da Bahia com mais solicitações deferidas é :\n' , deferidos_ba['interessado'].value_counts().idxmax())


O município da Bahia com mais solicitações deferidas é :
 Luís Eduardo Magalhães


In [212]:
# 4) Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

# Consultar os dados da Bahia com interessado 'Estado'
df_ba_estado = consulta_pvl('BA', 'Estado')

# Salvar o DataFrame como arquivo CSV
df_ba_estado.to_csv('solicitacoes_bahia_estado.csv', index=False)

df_ba_estado

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,40577,Estado,Bahia,29,BA,PVL02.008660/2019-37,Encaminhado à PGFN com manifestação técnica fa...,17944.104392/2019-46,2021-12-22T20:28:31Z,Operação contratual externa (com garantia da U...,Profisco,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,4.000000e+07,0,0,30/12/2021
1,18515,Estado,Bahia,29,BA,None,Deferido,19407.000040/2002-87,2002-06-21T03:00:00Z,Operação contratual interna,Preservação e recuperação ambiental,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.893800e+07,1,0,08/07/2002
2,64646,Estado,Bahia,29,BA,PVL02.000882/2023-98,Encaminhado à PGFN com manifestação técnica fa...,17944.102176/2023-42,2024-08-21T21:04:44Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Corporação Andina de Fomento,Dólar dos EUA,1.500000e+08,0,0,03/09/2024
3,18633,Estado,Bahia,29,BA,None,Deferido,19407.000073/2003-16,2004-01-30T02:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.150132e+08,1,0,18/03/2004
4,19016,Estado,Bahia,29,BA,00000.000000/2009-87,Encaminhado à PGFN com manifestação técnica fa...,17944.000485/2015-79,2015-11-03T02:00:00Z,Operação contratual interna (com garantia da U...,Aditivo contratual,Instituição Financeira Nacional,Banco do Brasil S/A,Real,5.627400e+08,1,0,09/11/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,18567,Estado,Bahia,29,BA,None,Deferido,19407.000053/2002-56,2002-06-17T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.039426e+06,1,0,03/07/2002
83,18571,Estado,Bahia,29,BA,None,Deferido,19407.000054/2002-09,2002-06-17T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.797522e+05,1,0,08/07/2002
84,18441,Estado,Bahia,29,BA,None,Deferido,19407.000023/2005-92,2006-01-16T02:00:00Z,Operação contratual interna,Desenvolvimento sustentável,Instituição Financeira Nacional,Banco do Nordeste do Brasil S/A,Dólar dos EUA,1.500000e+07,1,0,21/02/2006
85,18583,Estado,Bahia,29,BA,None,Deferido,19407.000057/2002-34,2002-06-17T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.303037e+06,1,0,05/07/2002
